<a href="https://colab.research.google.com/github/GayathriShrikanth/J046NLP/blob/master/Classwork/Sentiment_J046.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
df_twitter=pd.read_csv("https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv")

In [3]:
df_twitter.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...


In [4]:
import pandas as pd
import numpy as np
df=pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz", lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5,"A solid performer, and long time friend",1094169600,"09 3, 2004"
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007"
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011"
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006"
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013"


Polarity using TextBlob

In [0]:
from textblob import TextBlob
def text_blob(sentence):
  testimonial = TextBlob(sentence)
  return testimonial.polarity

In [0]:
df["Text_blob"]=df.reviewText[:100].apply(text_blob)

In [7]:
import nltk
nltk.download("vader_lexicon")
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [0]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sid = SIA()
def sentiment_nltk(str):
  sia = sid.polarity_scores(str)
  inverse = [(key,value) for value, key in sia.items()]
  (result,sen) = max(inverse)
  return result

In [0]:
df["nltk_sent"]=df.reviewText[:100].apply(sentiment_nltk)

In [13]:
#For a single sentence
sentiment_nltk(df.reviewText[0])

[(0.039, 'neg'), (0.781, 'neu'), (0.18, 'pos'), (0.9831, 'compound')]

In [16]:
#Polarity using textBlob and nltk on reviewText
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,Text_blob,nltk_sent
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5,"A solid performer, and long time friend",1094169600,"09 3, 2004",0.180000,0.9831
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007",0.170939,0.9963
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011",0.204091,0.7200
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006",0.214217,0.9970
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013",0.350000,0.9455


In [0]:
# On Twitter dataset
df_twitter["polarity"]=df_twitter.TweetText.apply(text_blob)

In [28]:
df_twitter.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText,polarity,twitter_sent
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...,0.000000,neutral
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...,0.500000,positive
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...,0.393750,positive
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...,0.541667,positive
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...,0.200000,positive


In [52]:
df_twitter = df_twitter[df_twitter['Sentiment'] != "irrelevant"]
df_twitter['Sentiment'] = pd.Categorical(df_twitter['Sentiment'])
df_twitter['Sentiment'].cat.categories

Index(['negative', 'neutral', 'positive'], dtype='object')

In [0]:
# Classify as positive negetive or neutral using polarity
def classify(pol):
  if pol>0:
    return "positive"
  elif pol<0:
    return "negative"
  elif pol==0:
    return "neutral"
  else:
    return "irrelevant"

In [0]:
df_twitter["twitter_sent"]=df_twitter.polarity.apply(classify)

In [55]:
df_twitter.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText,polarity,twitter_sent
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...,0.000000,neutral
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...,0.500000,positive
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...,0.393750,positive
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...,0.541667,positive
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...,0.200000,positive


In [56]:
from sklearn.metrics import confusion_matrix
confusion_matrix(df_twitter["Sentiment"], df_twitter["twitter_sent"])

array([[ 234,  171,  167],
       [ 201, 1213,  919],
       [  38,  124,  357]])

In [57]:
from sklearn.metrics import classification_report
print(classification_report(df_twitter["Sentiment"], df_twitter["twitter_sent"]))

              precision    recall  f1-score   support

    negative       0.49      0.41      0.45       572
     neutral       0.80      0.52      0.63      2333
    positive       0.25      0.69      0.36       519

    accuracy                           0.53      3424
   macro avg       0.52      0.54      0.48      3424
weighted avg       0.67      0.53      0.56      3424



Glove

In [39]:
!pip install glove_python

     |████████████████████████████████| 266kB 2.7MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=700353 sha256=1483c4e9cff32048e849127ca3c949d370057fd452575b204b7bb4616fd2392f
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [0]:
import glove
from glove import Corpus, Glove

In [41]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [44]:
Tweets = df_twitter["TweetText"].apply(nltk.word_tokenize)
corpus = Corpus()
corpus.fit(Tweets,window=10)
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)


Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [0]:
glove.add_dictionary(corpus.dictionary)

In [46]:
glove.most_similar('bad')

[('email', 0.9586292819504754),
 ('real', 0.9428511168190109),
 ('using', 0.9371486508187631),
 ('tweet', 0.9353965535030802)]